In [130]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup


In [131]:
url = "https://www.ocasionplus.com/coches-ocasion?type%3DCAR%26sort%3DFECHAOP_DESC%26page%3D2"

In [132]:
llamada = requests.get(url)  
llamada

<Response [200]>

In [133]:
res_coches = requests.get(url)

In [134]:
sopa_coches = BeautifulSoup(res_coches.content, "html.parser")


In [135]:
# lista_nombres_coche = sopa_coches.find_all("h2")[1:]

# lista_nombres_coche[:3]

In [136]:
#COLUMNA NOMBRES 

# lista_nombre_coches_2 = []

# for nombre in lista_nombres_coche:
#     lista_nombre_coches_2.append(nombre.getText())
    

# lista_nombre_coches_2

In [137]:
#PRECIO ORIGINAL 

# lista_precio_original = sopa_coches.find_all("span", {"class": "jsx-2870186023 priceLast"})
# lista_precio_original[:4]

In [138]:
# precios_originales = []

# for precio in lista_precio_original:
#      precios_originales.append(precio.text)


# precios_originales[:4]

In [139]:
#PRECIO REBAJADO 

# lista_precio_rebajado = sopa_coches.find_all("span", {"class": "jsx-2689423372 price"})
# lista_precio_rebajado[:4]

In [140]:
# precios_rebajados = []

# for precio in lista_precio_rebajado:
#      precios_rebajados.append(precio.text)


# precios_rebajados[:4]

#MONTAMOS LA ESTUCTURA
- Coche
- Precio rebajado
- Precio original
- Kilómetros


In [141]:
#INFO QUE EXTRAEMOS DE LA WEB 
def scrape_coches_ocasion(url):
    res_coches = requests.get(url)
    print(f"La respuesta a tu petición ha sido de {res_coches.status_code}")
    sopa_coches = BeautifulSoup(res_coches.content, "html.parser")


    #Nombre
    lista_nombres_coche = sopa_coches.find_all("h2", {"style": "overflow-x:hidden"})
    lista_nombre_coches_2 = []
    for nombre in lista_nombres_coche:
        lista_nombre_coches_2.append(nombre.getText())
    

    #Precio original + Precio Rebajado 

    lista_precios_all = sopa_coches.find_all("div", {"class": "jsx-689670945 priceContainer"})
    precios_originales = []
    precios_rebajados = []

    for precio in lista_precios_all:
        precio_text = precio.text
        precio_parts = precio_text.split("€")
        
        if len(precio_parts) == 3:
            precio_original = precio_parts[0].strip()
            precio_descuento = precio_parts[1].strip()
            precios_originales.append(precio_original)
            precios_rebajados.append(precio_descuento)
        elif len(precio_parts) == 2:
            precio_original = precio_parts[0].strip()
            precios_originales.append(precio_original)
            precios_rebajados.append("None")

        
    #Kilometros
    extraccion_kms = sopa_coches.find_all("span", {"class": "jsx-3960002075 detail big"})
    lista_kms_coche = []
    for elemento in extraccion_kms:
        text = elemento.getText()
        if "km" in text:
            lista_kms_coche.append(text)

    #Creamos diccionario donde guardaremos las listas con la informacion: 
    diccionario_coches = {"Coche": [], "Precio Rebajado": [], "Precio Original": [], "Kilómetros": []}

    diccionario_coches["Coche"] = lista_nombre_coches_2
    diccionario_coches["Precio Rebajado"] = precios_rebajados
    diccionario_coches["Precio Original"]= precios_originales
    diccionario_coches["Kilómetros"] = lista_kms_coche
    

    df_coches = pd.DataFrame(diccionario_coches)
            
    return df_coches


In [142]:
scrape_coches_ocasion("https://www.ocasionplus.com/coches-ocasion?type%3DCAR%26sort%3DFECHAOP_DESC%26page%3D2")

La respuesta a tu petición ha sido de 200


,Coche,Precio Rebajado,Precio Original,Kilómetros
0,Chevrolet Aveo 1.4 16v LT AUTO (101 CV),None,5.450,110.475 km
1,BMW Serie 3 320d Auto (184 CV),None,18.300,128.100 km
2,Citroen C-Elysee PureTech 82 Shine (82 CV),11.227,12.350,14.925 km
3,Smart ForFour ForFour 1.0 (71 CV),10.181,11.200,66.933 km
4,Peugeot 2008 SUV SUV PureTech 130 S&S Allure (...,11.454,12.700,131.603 km
5,Toyota Yaris 1.0 Active (72 CV),11.090,12.350,81.009 km
6,Renault Captur Limited TCe (130 CV) GPF,12.545,13.800,109.395 km
7,SEAT Leon 1.5 eTSI S&S FR DSG (150 CV),22.718,24.990,27.566 km
8,BMW X1 sDrive18d 2.0 (150 CV),17.400,18.450,140.870 km
9,Ford S-Max 2.0 TDCI Titanium PowerShift AUTO (...,None,15.400,171.644 km


In [143]:
df_coches = pd.DataFrame(diccionario_coches)

NameError: name 'diccionario_coches' is not defined

In [ ]:
# lista_precios_all = sopa_coches.find_all("div", {"class": "jsx-689670945 priceContainer"})
# precios_originales = []
# precios_rebajados = []

# for precio in lista_precios_all:

#     if len(lista_precios_all[0].text.split("€")[:3]) == 3: 
#         precio_0 = precio.text.split("€")[0].strip()
#         precios_originales.append(precio_0)
#         precio_1 = precio.text.split("€")[1].strip()
#         precios_rebajados.append(precio_1)
#     elif len(lista_precios_all[0].text.split("€")[:3]) == 2:
#         precio_0 = precio.text.split("€")[0].strip()
#         precios_originales.append(precio_0)
#         precio_1 = precio.text.split("€")[0].strip()
#         precios_rebajados.append("none")

In [ ]:
# lista_precios_all = sopa_coches.find_all("div", {"class": "jsx-689670945 priceContainer"})
# precios_originales = []
# precios_rebajados = []

# for precio in lista_precios_all:
#     precio_text = precio.text
#     precio_parts = precio_text.split("€")
    
#     if len(precio_parts) == 3:
#         precio_original = precio_parts[0].strip()
#         precio_descuento = precio_parts[1].strip()
#         precios_originales.append(precio_original)
#         precios_rebajados.append(precio_descuento)
#     elif len(precio_parts) == 2:
#         precio_original = precio_parts[0].strip()
#         precios_originales.append(precio_original)
#         precios_rebajados.append("None")




In [ ]:
#lista_precio_original[0].text.split("€")[:3]

In [ ]:
scrape_coches_ocasion("https://www.ocasionplus.com/coches-ocasion?type%3DCAR%26sort%3DFECHAOP_DESC%26page%3D2")